<a href="https://colab.research.google.com/github/devyadav11/DL_assignment_2/blob/main/DLAssignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

Using device: cuda
GPU Name: Tesla T4


In [2]:
# !pip install opencv-python
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
from sklearn.utils import shuffle # for shuffling
import os
import cv2
import random
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image



from sklearn.model_selection import train_test_split

In [3]:
!pip install wandb
import wandb
# !wandb login
wandb.login(key="500727e3b54d5202c75cd3136d6326f29e89e05f")

#key = 500727e3b54d5202c75cd3136d6326f29e89e05f

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ee23m074 (devyadav11) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
 !wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip


--2025-04-17 12:20:02--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.207, 142.250.101.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G   162MB/s    in 31s     

2025-04-17 12:20:33 (117 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [5]:
 !unzip -q nature_12K.zip

In [6]:
 !rm nature_12K.zip

In [7]:
# dataset_path = "/content/inaturalist_12K"

In [7]:
image_hight = 256
image_width = 256

In [8]:
def load_data(batchSize):
  transform_data = transforms.Compose([ transforms.Resize((image_hight, image_width)), transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ])   # normalise = output = (input - mean) / std ) (convert to (-1 to 1)

  train_data = datasets.ImageFolder(root = "/content/inaturalist_12K/train", transform=transform_data)

  class_index = train_data.class_to_idx

  training_idx = []
  val_idx  = []

  for train_class, train_index in class_index.items():
    index = []
    for k, (img_path, lable) in enumerate(train_data.samples):
      if lable == train_index:
        index.append(k)


    train_indices, val_indices = train_test_split(index, test_size=0.2, random_state=42)
    training_idx.extend(train_indices)           # extend creates a single flat list
    val_idx.extend(val_indices)

  training_data = Subset(train_data, training_idx)
  val_data = Subset(train_data, val_idx)




  train_loader = DataLoader(training_data, batch_size=batchSize, shuffle=True, pin_memory= True )
  val_loader = DataLoader(val_data, batch_size=batchSize, shuffle=True, pin_memory= True)

  test_data = datasets.ImageFolder(root = "/content/inaturalist_12K/val", transform=transform_data)
  test_laoder = DataLoader(test_data, batch_size=batchSize, shuffle=True, pin_memory= True)



  return train_loader, val_loader, test_laoder






















  #return  class_index




In [9]:
def find_accuracy(model, criterion, dataLoader, dataName):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataLoader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f'{dataName} Loss: {val_loss/len(dataLoader)},'
          f'{dataName} Accuracy: {100*correct/total}%\n')

    #wandb.log({f'{dataName}_accuracy': 100*correct/total})
    #wandb.log({f'{dataName}_loss': val_loss/len(dataLoader)})

In [10]:
class ConvNN(nn.Module):
  def __init__(self, num_filters, size_filter, activation_fxn, num_neurons_dense, batch_norm, dropout_prob ):
    super(ConvNN, self).__init__()

    layers = []

    for i in range(len(num_filters)):
      if i == 0:
        layers.append(nn.Conv2d(in_channels=3, out_channels=num_filters[i], kernel_size=size_filter[i], stride=1, padding=0))
        layers.append(activation_fxn)
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        if batch_norm == 'true':
          layers.append(nn.BatchNorm2d(num_features=num_filters[i]))
      else:
        layers.append(nn.Conv2d(in_channels=num_filters[i-1], out_channels=num_filters[i], kernel_size=size_filter[i], stride = 1, padding = 0))
        layers.append(activation_fxn)
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        if batch_norm == 'true':
          layers.append(nn.BatchNorm2d(num_features=num_filters[i]))

      self.conv_stack = nn.Sequential(*layers)
      flattened_tensor = torch.flatten(self.conv_stack(torch.zeros(1, 3, image_hight, image_width)))
      dim_lastConv = flattened_tensor.shape[-1]
      self.flattened = nn.Flatten()
      self.dense_layer = nn.Linear(in_features=dim_lastConv, out_features=num_neurons_dense)
      self.dropout = nn.Dropout(dropout_prob)
      self.output_layer = nn.Linear(in_features=num_neurons_dense, out_features=10)

  def forward(self, x):
    x = self.conv_stack(x)
    x = self.flattened(x)
    x = self.dense_layer(x)
    x = self.dropout(x)
    x = self.output_layer(x)

    return x



def modal_training(optimiser_fn, num_epochs, batch_size, num_filters, size_filter,
                   activation_fxn, num_neurons_dense, batch_norm, dropout_prob, weight_decay, learning_rate):


   # Initialize wandb run
  #wandb.init(project="assignment_2")

  train_loader, val_loader, test_loader = load_data(batch_size)

  if activation_fxn == "relu":
    activation_fxn = nn.ReLU()
  elif activation_fxn == "leaky_relu":
    activation_fxn = nn.LeakyReLU()
  elif activation_fxn == "elu":
    activation_fxn = nn.ELU()
  else:
    activation_fxn = nn.Sigmoid()


  model = ConvNN(num_filters, size_filter, activation_fxn, num_neurons_dense, batch_norm, dropout_prob)

  criterion = nn.CrossEntropyLoss()

  if optimiser_fn == "adam":
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  elif optimiser_fn == "nadam":
    optimizer = optim.NAdam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  elif optimiser_fn == "rmsprop":
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  else:
 # stocastic gradient decent
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)




  model = torch.nn.DataParallel(model,device_ids = [0]).to(device)

  for epoch in range(num_epochs):
    model.train()

    for index, (inputs, labels) in enumerate(tqdm(train_loader, desc=f'Training Progress {epoch+1}')):
      inputs, labels = inputs.to(device), labels.to(device)                        # keeps input and lables run on same GPU, or CPU if u choose multiple devices
      optimizer.zero_grad()                                                        # set gradients to zero
      outputs = model(inputs)                                                      # forward pass
      loss = criterion(outputs, labels)                                            # calculate loss
      loss.backward()                                                              # backward pass
      optimizer.step()                                                             # updates the parameters after back prop


    find_accuracy(model, criterion, train_loader, "train")
    find_accuracy(model, criterion, val_loader, "validation")


  return model















In [12]:
# modal = modal_training(optimiser_fn = "adam" , num_epochs = 10, batch_size = 64, num_filters = [32,32,32,32,32], size_filter = [3,5,3,5,3],
#                    activation_fxn = "elu", num_neurons_dense = 512, batch_norm = False, dropout_prob = 0.2, weight_decay = 0.005, learning_rate = 1e-4)


In [ ]:
def main():
    wandb.init(project="assignment_2")
    config = wandb.config
    run_name = f"{config.optimiser}_{config.activation}_{config.num_filters}_{config.batch_size}"

    # Set the run name
    wandb.run.name = run_name
    wandb.run.save()

    # Define and train the model as before
    modal_training(learning_rate = config.learning_rate, num_filters = config.num_filters,
                size_filter = config.filter_sizes, activation_fxn = config.activation,
                optimiser_fn = config.optimiser, num_neurons_dense = config.dense_layer,
                weight_decay = config.weight_decay, dropout_prob = config.dropout, batch_norm = False,
                batch_size = config.batch_size, num_epochs = 10)

sweep_config = {
    'method': 'bayes',
    'name' : 'sweep cross entropy',
    'metric': {
      'name': 'validation_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'num_filters': {
          'values': [[32,32,32,32,32],[32,64,64,128,128],[128,128,64,64,32],[32,64,128,256,512],[128,128,128,128,128],[32,32,64,64,32,32,128,32]]
        },
        'filter_sizes': {
          'values': [[3,3,3,3,3], [5,5,5,5,5], [3,5,3,5,3]]
        },
        'weight_decay': {
            'values':[0, 0.0005, 0.5]
        },
        'learning_rate': {
            'values':[1e-3,1e-4]
        },
        'weight_decay': {
            'values': [0, 0.0005, 0.005]
        },
        'dropout': {
            'values': [0, 0.2, 0.4]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'activation': {
            'values': ['relu', 'elu', 'selu']
        },
        'optimiser': {
            'values': ['nadam', 'adam', 'rmsprop']
        },
        'batch_norm':{
            'values': ['true','false']
        },
        'batch_size': {
            'values': [32, 64]
        },
        'dense_layer':{
            'values': [128, 256, 512]
        }
    }
}


sweep_id = wandb.sweep(sweep=sweep_config,project='assignment_2')
wandb.agent(sweep_id , function = main , count = 100)

wandb.finish()

Create sweep with ID: 257wsuhy
Sweep URL: https://wandb.ai/devyadav11/assignment_2/sweeps/257wsuhy


wandb: Agent Starting Run: e4yn07ux with config:
wandb: 	activation: elu
wandb: 	batch_norm: true
wandb: 	batch_size: 32
wandb: 	dense_layer: 512
wandb: 	dropout: 0.4
wandb: 	filter_sizes: [3, 5, 3, 5, 3]
wandb: 	learning_rate: 0.001
wandb: 	num_filters: [128, 128, 64, 64, 32]
wandb: 	optimiser: nadam
wandb: 	weight_decay: 0


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
Training Progress 1: 100%|██████████| 250/250 [01:50<00:00,  2.27it/s]


train Loss: 2.1732629556655882,train Accuracy: 24.56557069633704%

validation Loss: 2.2282145212567044,validation Accuracy: 22.6%



Training Progress 2: 100%|██████████| 250/250 [01:31<00:00,  2.72it/s]


...

Training Progress 2: 100%|██████████| 250/250 [01:28<00:00,  2.82it/s]


train Loss: 2.13103692483902,train Accuracy: 23.85298162270284%

validation Loss: 2.1329177163896107,validation Accuracy: 24.5%



Training Progress 3: 100%|██████████| 250/250 [01:28<00:00,  2.81it/s]


train Loss: 2.058082466125488,train Accuracy: 25.778222277784725%

validation Loss: 2.078570659198458,validation Accuracy: 25.5%



Training Progress 4: 100%|██████████| 250/250 [01:29<00:00,  2.78it/s]


In [ ]:
# my_list = []
# f1 = my_list.append(nn.Softmax(dim = 1))
# f2 = my_list.append(nn.ReLU())
# f3 = my_list.append(nn.LeakyReLU())
# f4 = my_list.append(nn.Softmax(dim = 1))
# x = [1, 2, 3]
# #my_list.extend([4, 5, 6])
# #my_list.append([7, 8, 9])

# m = nn.Sequential(*my_list)
# print(my_list)
# z = m(x)

In [ ]:
# train_loader, val_loader, test_loader = load_data(1000)
# for index, (inputs, labels) in enumerate(train_loader):
#   print(index, inputs, labels)

In [ ]:
# transform = transforms.Compose([ transforms.Resize((256, 256)), transforms.ToTensor() ])
